In [1]:
import ortools
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

### 条件設定

In [2]:
MAX_DAY = 12 # ジムに行きたい日数
MAX_MENU_BY_DAY = 6 # 1日あたりにこなせる最大メニュー数

REST_DAY_OF_WEEK = [3,6] #休みたい曜日 (月曜:1, 火曜:2, ...)
REST_DAY = [11, 19] #休みたい日

# 休みたい曜日を休みたい日をまとめる
for rest_day in REST_DAY_OF_WEEK:
    for week in range(4):
        REST_DAY.append(rest_day + 7*week)

In [3]:
# 部位ごとの回復日数
recover_times = {
    '胸':2,
    '背中':3,
    '腕':2,
    '肩':2,
    '腹筋':1,
    '脚':5
}
# 優先度
importance = {
    '胸':1.2,
    '背中':2,
    '腕':1,
    '肩':1.5,
    '腹筋':1,
    '脚':2.5
}
# 部位ごとのメニュー数
num_menu = {
    '胸':3,
    '背中':2,
    '腕':2,
    '肩':2,
    '腹筋':1,
    '脚':4
}

In [4]:
df = pd.DataFrame([recover_times.values(), importance.values(), num_menu.values()]).T
df.columns = ['回復期間', '優先度', 'メニュー数']
df.index = recover_times.keys()
df['回復期間'] = df['回復期間'].astype(int)
df['メニュー数'] = df['メニュー数'].astype(int)
df

,回復期間,優先度,メニュー数
胸,2,1.2,3
背中,3,2.0,2
腕,2,1.0,2
肩,2,1.5,2
腹筋,1,1.0,1
脚,5,2.5,4


### ソルバオブジェクト・変数の定義

In [5]:
# ソルバオブジェクト作成
solver = pywraplp.Solver("MIP", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [6]:
# 決定変数(部位トレ)
x = {}
for target, recover_time in recover_times.items():
    x[target] = [[solver.BoolVar('{}_{}_{}'.format(target, weeks, weekday)) for weekday in range(7)] for weeks in range(4)]
    x[target] = np.array(x[target]).flatten()

# 補助変数(ジム行く日)
go_gym = [[solver.BoolVar('go_gym_{}_{}'.format(weeks, weekday)) for weekday in range(7)] for weeks in range(4)]
go_gym = np.array(go_gym).flatten()

### 制約条件

In [7]:
# トレーニングする日はジムに行く ※家トレは許されない
for target, recover_time in recover_times.items():
    for day in range(len(go_gym)):
        solver.Add(go_gym[day] >= x[target][day])

In [8]:
# ジム行く日数の制限
solver.Add(np.sum(go_gym) <= MAX_DAY)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0xffff74af9c30> >

In [9]:
# 1日あたりのメニュー数に制限 　※オーバーワークは敵
for day in range(len(go_gym)):
    solver.Add(np.sum([
        x[target][day] * num_menu[target] for target in x.keys()
    ]) <= MAX_MENU_BY_DAY)

In [10]:
# トレーニング後、回復期間以内にトレーニングしない ※超回復理論を信じろ
for target, recover_time in recover_times.items():
    for i,start_point in enumerate(range(len(x[target]) - recover_time)):
        solver.Add(np.sum([x[target][start_point + i] for i in range(recover_time + 1)]) <= 1)

In [11]:
# 休みたい日が設定されている場合、その日はジムに行かない
for rest_day in REST_DAY:
    solver.Add(go_gym[rest_day] == 0)

### 目的関数

In [12]:
# 目的関数はこなすメニュー数の最大化
#solver.Maximize(np.sum([x.values() * ])) 
solver.Maximize(np.sum([x[target] * importance[target] for target in recover_times.keys()]))

### 最適解の計算

In [13]:
# ソルバに最適解を計算してもらう
status = solver.Solve()
if status == solver.OPTIMAL:
    print("solved")

solved


In [14]:
menu_by_day = ['' for _ in range(len(go_gym))] # 日ごとのトレーニング部位
count_day_for_target = {target: 0 for target in recover_times.keys()} # 部位ごとのトレーニング日数

# 日ごとのトレーニング部位と部位ごとのトレーニング日数を計上
for day in range(len(go_gym)):
    for target in recover_times.keys():
        if x[target][day].solution_value() == 1:
            menu_by_day[day] += target + '  '
            count_day_for_target[target] += 1

In [15]:
# display用css
body_css = {"background-color": "#eee"}
col_css = {"background-color": "#ddd"}

# 日ごとのトレーニング部位
cal_df = pd.DataFrame(np.array(menu_by_day).reshape(4,7) , 
                   columns=['月','火','水','木','金','土','日'],
                   index=['1週目','2週目','3週目','4週目'])
cal_df = cal_df.style.set_properties(subset=cal_df.columns, **{'width': '80px'}) # セル幅の調整
cal_df = cal_df.set_properties( **body_css)
cal_df = cal_df.set_properties(subset=['月', '水', '金', '日'], **col_css)
display(cal_df)

# 部位ごとのトレーニング日数
count_df = pd.DataFrame(count_day_for_target.values(), index=count_day_for_target.keys(), columns=['日数'])
display(count_df.T)

,月,火,水,木,金,土,日
1週目,胸 背中 腹筋,肩 脚,,,背中 腕 肩,,
2週目,肩 脚,胸 背中 腹筋,,,,背中 肩 腹筋,
3週目,,胸 肩 腹筋,背中 脚,,胸 肩 腹筋,,
4週目,背中 腕 肩,,,,肩 脚,胸 背中 腹筋,


,胸,背中,腕,肩,腹筋,脚
日数,5,7,2,8,6,4


In [16]:
def func(x):
    return x.solution_value()

In [17]:
print("ジム行く日")
display(pd.DataFrame(go_gym.reshape(4,7)).applymap(func))

for target in recover_times.keys():
    print(target + "DAY")
    display(pd.DataFrame(x[target].reshape(4,7)).applymap(func) * num_menu[target])

ジム行く日


,0,1,2,3,4,5,6
0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0


胸DAY


,0,1,2,3,4,5,6
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,3.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,3.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,3.0,0.0


背中DAY


...

,0,1,2,3,4,5,6
0,0.0,2.0,0.0,0.0,2.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0,2.0,0.0
2,0.0,2.0,0.0,0.0,2.0,0.0,0.0
3,2.0,0.0,0.0,0.0,2.0,0.0,0.0


腹筋DAY


,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0


脚DAY


,0,1,2,3,4,5,6
0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0
